# Программирование на языке Python
## Семинар 19. Работа с файлами разных форматов

https://historik.val.se/val/val2018/slutresultat/R/rike/index.html

**Жирный текст**

<b>Жирный текст</b>

<p>
    Текст<br>пункт 1,<br><u style="color: red;">
        пункт 2
    </u>
</p>

<table>
    <thead>
        <tr>
            <th>Заголовок 1</th>
            <th>Заголовок 2</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="background-color: hotpink;">Ячейка 1.1</td>
            <td>Ячейка 2.1</td>
        </tr>
        <tr>
            <td>Ячейка 2.1</td>
            <td>Ячейка 2.2</td>
        </tr>
    </tbody>
</table>

In [1]:
import requests
from bs4 import BeautifulSoup, Tag

import numpy as np
import pandas as pd

In [24]:
# url = 'https://historik.val.se/val/val2018/slutresultat/R/kommun/10/82/index.html'
url = 'https://historik.val.se/val/val2018/slutresultat/R/rike/index.html'

page = requests.get(url)
page_text = page.content.decode()
soup = BeautifulSoup(page_text)

page.close()

In [26]:
nt = get_navigation_table(soup)

In [31]:
nt_gen = parse_navigation_table(nt)

In [32]:
nt_iter = iter(nt_gen)

In [21]:
check[2].a.get('href')

'../../../../../slutresultat/R/kommun/10/82/index.html'

In [79]:
h2 = soup.find('h2', {'id': ''})

In [3]:
import requests
from bs4 import BeautifulSoup, Tag
from warnings import warn


INDICES = {0, 1, 2, 6}
MINLEVELS = {'valdistrikt', 'onsdagsdistrikt'}
MAIN_TABLE_CLASS = 'sorteringsbar_tabell'
NAVIGATION_TABLE_ID = 'oversiktstabell'
URL = 'https://historik.val.se/val/val2018/slutresultat/R'


def get_table(page: BeautifulSoup) -> Tag:
    table = page.find('table', {'class': MAIN_TABLE_CLASS})

    return table


def parse_table(table: Tag) -> dict:
    # extract headers
    keys = [th.text for index, th in enumerate(table.find_all('th')) if index in INDICES]

    # extract values
    rows = table.find_all('tr')[1:]
    rows_text = [[td.text for index, td in enumerate(row.find_all('td')) if index in INDICES] for row in rows]

    # get dict
    table_dict = dict(zip(keys, zip(*rows_text)))

    return table_dict


def get_page(url: str) -> BeautifulSoup:
    response = requests.get(url)
    status_code = response.status_code

    if status_code == 200:
        # get content
        content = response.content.decode()  # response.text
    
        # parse content
        page = BeautifulSoup(content)
    else:
        warn(f'Page for url {url} not parsed, status code is {status_code}, returning None')
        page = None

    response.close()

    return page


def get_navigation_table(page: BeautifulSoup) -> Tag:
    navigation_table = page.find('table', {'id': NAVIGATION_TABLE_ID})

    return navigation_table


def parse_navigation_table(navigation_table: Tag):
    tds = navigation_table.find_all('td')
    active_index = [index for index, td in enumerate(tds) if td.get('class') == ['aktiv']][0]

    for td in tds[(active_index + 1):]:
        a = td.a
        
        # get level
        level = td.parent.get('class')[0]

        # get district
        district = a.text

        # get page
        href = a.get('href')
        url = f'{URL}/{href[(href.find("R") + 2):]}'
        page = get_page(url)

        if page is None:
            continue

        yield level, district, page


def parse_page(page: BeautifulSoup, districts=[]):
    navigation_table = get_navigation_table(page)
    navigation_table_parsed = parse_navigation_table(navigation_table)
    indent = '\t' * len(districts)

    for level, district, page in navigation_table_parsed:
        if level in MINLEVELS:
            table = get_table(page)
            table_parsed = parse_table(table)
            TABLES.append((districts + [(level, district)], table_parsed))

            print(f'{indent}({level}) {district} table parsed successfully...')
        else:
            print(f'{indent}({level}) {district}')
            parse_page(page, districts + [(level, district)])

In [4]:
url_main = 'https://historik.val.se/val/val2018/slutresultat/R/rike/index.html'
page = get_page(url_main)

TABLES = []
parse_page(page)

(riksdagsvalkrets) Blekinge län
	(kommun) Karlshamn
		(valdistrikt) Centrala Asarum table parsed successfully...
		(valdistrikt) Froarp table parsed successfully...
		(valdistrikt) Gustavstorp table parsed successfully...
		(valdistrikt) Horsaryd table parsed successfully...
		(valdistrikt) Hällaryd table parsed successfully...
		(valdistrikt) Högadal table parsed successfully...
		(valdistrikt) Korpadalen table parsed successfully...
		(valdistrikt) Mörrum västra table parsed successfully...
		(valdistrikt) Mörrum östra table parsed successfully...
		(valdistrikt) Prästslätten table parsed successfully...
		(valdistrikt) Skogsborg table parsed successfully...
		(valdistrikt) Stadsporten table parsed successfully...
		(valdistrikt) Svängsta nordöst table parsed successfully...
		(valdistrikt) Svängsta sydväst table parsed successfully...
		(valdistrikt) Torget table parsed successfully...
		(valdistrikt) Tubbaryd table parsed successfully...
		(valdistrikt) Vägga table parsed successfu

KeyboardInterrupt: 

In [6]:
import pickle

with open('tmp/tables.pickle', 'rb') as infile:
    TABLES = pickle.load(infile)

In [18]:
import numpy as np
import pandas as pd

def process_value_pd(table_dict) -> pd.DataFrame:
    table_df = pd.DataFrame(table_dict)

    return table_df

def process_value_np(table_dict) -> np.ndarray:
    table_np = np.array(list(table_dict.values()))

    return table_np

In [27]:
%%timeit

values_df = pd.concat([process_value_pd(item[-1]) for item in TABLES])

320 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit

values_nps = [process_value_np(item[-1]) for item in TABLES]
values_df2 = pd.DataFrame(np.hstack(values_nps).T, columns=['Förk.', 'Parti', 'Antal2018', 'Antal2014'])

72.8 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
pd.DataFrame(np.hstack(values_nps).T)

,0,1,2,3
0,M,Moderaterna,162,164
1,C,Centerpartiet,37,28
2,L,Liberalerna (tidigare Folkpartiet),37,19
3,KD,Kristdemokraterna,53,29
4,S,Arbetarepartiet-Socialdemokraterna,519,604
...,...,...,...,...
100553,ÖVR,Övriga anmälda partier,7,6
100554,,Giltiga röster,359,219
100555,OGEJ,Ogiltiga röster - inte anmälda partier,,
100556,BLANK,Ogiltiga röster - blanka,1,2


In [20]:
values_nps

[array([['M', 'C', 'L', 'KD', 'S', 'V', 'MP', 'SD', 'FI', 'ÖVR', '\xa0',
         'OGEJ', 'BLANK', 'OG', 'VDT', '\xa0'],
        ['Moderaterna', 'Centerpartiet',
         'Liberalerna (tidigare Folkpartiet)', 'Kristdemokraterna',
         'Arbetarepartiet-Socialdemokraterna', 'Vänsterpartiet',
         'Miljöpartiet de gröna', 'Sverigedemokraterna',
         'Feministiskt initiativ', 'Övriga anmälda partier',
         'Giltiga röster', 'Ogiltiga röster - inte anmälda partier',
         'Ogiltiga röster - blanka', 'Ogiltiga röster - övriga',
         'Valdeltagande', 'Antal röstberättigade'],
        ['162', '37', '37', '53', '519', '82', '32', '311', '8', '11',
         '1252', '\xa0', '13', '\xa0', '1265', '1555'],
        ['164', '28', '19', '29', '604', '60', '54', '217', '19', '6',
         '1200', '\xa0', '10', '\xa0', '1210', '1491']], dtype='<U38'),
 array([['M', 'C', 'L', 'KD', 'S', 'V', 'MP', 'SD', 'FI', 'ÖVR', '\xa0',
         'OGEJ', 'BLANK', 'OG', 'VDT', '\xa0'],
        ['

In [17]:
np.array(list(TABLES[0][-1].values()))

array([['M', 'C', 'L', 'KD', 'S', 'V', 'MP', 'SD', 'FI', 'ÖVR', '\xa0',
        'OGEJ', 'BLANK', 'OG', 'VDT', '\xa0'],
       ['Moderaterna', 'Centerpartiet',
        'Liberalerna (tidigare Folkpartiet)', 'Kristdemokraterna',
        'Arbetarepartiet-Socialdemokraterna', 'Vänsterpartiet',
        'Miljöpartiet de gröna', 'Sverigedemokraterna',
        'Feministiskt initiativ', 'Övriga anmälda partier',
        'Giltiga röster', 'Ogiltiga röster - inte anmälda partier',
        'Ogiltiga röster - blanka', 'Ogiltiga röster - övriga',
        'Valdeltagande', 'Antal röstberättigade'],
       ['162', '37', '37', '53', '519', '82', '32', '311', '8', '11',
        '1252', '\xa0', '13', '\xa0', '1265', '1555'],
       ['164', '28', '19', '29', '604', '60', '54', '217', '19', '6',
        '1200', '\xa0', '10', '\xa0', '1210', '1491']], dtype='<U38')

In [11]:
values_dfs

,Förk.,Parti,Antal2018,Antal2014
0,M,Moderaterna,162,164
1,C,Centerpartiet,37,28
2,L,Liberalerna (tidigare Folkpartiet),37,19
3,KD,Kristdemokraterna,53,29
4,S,Arbetarepartiet-Socialdemokraterna,519,604
...,...,...,...,...
9,ÖVR,Övriga anmälda partier,7,6
10,,Giltiga röster,359,219
11,OGEJ,Ogiltiga röster - inte anmälda partier,,
12,BLANK,Ogiltiga röster - blanka,1,2


In [8]:
TABLES[0]

([('riksdagsvalkrets', 'Blekinge län'),
  ('kommun', 'Karlshamn'),
  ('valdistrikt', 'Centrala Asarum')],
 defaultdict(list,
             {'Förk.': ['M',
               'C',
               'L',
               'KD',
               'S',
               'V',
               'MP',
               'SD',
               'FI',
               'ÖVR',
               '\xa0',
               'OGEJ',
               'BLANK',
               'OG',
               'VDT',
               '\xa0'],
              'Parti': ['Moderaterna',
               'Centerpartiet',
               'Liberalerna (tidigare Folkpartiet)',
               'Kristdemokraterna',
               'Arbetarepartiet-Socialdemokraterna',
               'Vänsterpartiet',
               'Miljöpartiet de gröna',
               'Sverigedemokraterna',
               'Feministiskt initiativ',
               'Övriga anmälda partier',
               'Giltiga röster',
               'Ogiltiga röster - inte anmälda partier',
               'Ogiltiga rö

In [73]:
t = parse_table(table)

In [74]:
pd.DataFrame(t)

,Förk.,Parti,Antal2018,Antal2014
0,M,Moderaterna,3632,3923
1,C,Centerpartiet,1392,1015
2,L,Liberalerna (tidigare Folkpartiet),727,616
3,KD,Kristdemokraterna,1095,647
4,S,Arbetarepartiet-Socialdemokraterna,7073,8114
5,V,Vänsterpartiet,1306,1044
6,MP,Miljöpartiet de gröna,718,1192
7,SD,Sverigedemokraterna,5218,3867
8,FI,Feministiskt initiativ,95,430
9,ÖVR,Övriga anmälda partier,183,144
